### Get the Kafka cluster ID
This is required to create new topics

In [ ]:
import requests 

sasquatch_rest_proxy_url = "https://data-dev.lsst.cloud/sasquatch-rest-proxy"

headers = {"content-type": "application/json"}

r = requests.get(f"{sasquatch_rest_proxy_url}/v3/clusters", headers=headers)

cluster_id = r.json()['data'][0]['cluster_id']

print(cluster_id)

### Create a new topic
REST Proxy at data-dev is configured to work with topics in the `lsst.Test` namespace

In [ ]:
namespace = "lsst.Test"
topic_name = "topic1"

topic_config = {  
        "topic_name": f"{namespace}.{topic_name}",
        "partitions_count": 1,
        "replication_factor": 3
    }

headers = {"content-type": "application/json"}

response = requests.post(f"{sasquatch_rest_proxy_url}/v3/clusters/{cluster_id}/topics", json=topic_config, headers=headers)
print(response.text)

### Send some data
Create a payload based on the example in https://sasquatch.lsst.io/user-guide/restproxy.html#kafka-rest-proxy

In [ ]:
import json

value_schema = json.dumps({"namespace": f"{namespace}", 
                           "type": "record", 
                           "name": f"{topic_name}", 
                           "fields": [{"name": "timestamp", "type": "long"}, 
                                      {"name": "band", "type": "string"}, 
                                      {"name": "instrument", "type": "string", "default": "LSSTCam-imSim"}, 
                                      {"name": "meanSky","type": "float"}, 
                                      {"name": "stdevSky","type": "float"}]
                          })

payload = {
    "value_schema": value_schema,
    "records": [
        {
            "value": {
                "timestamp": 1681248783000000,
                "band": "y",
                "instrument": "LSSTCam-imSim",
                "meanSky": -213.75839364883444,
                "stdevSky": 2328.906118708811,
            }
        }
    ],
}
payload

In [ ]:
import requests

url = f"{sasquatch_rest_proxy_url}/topics/{namespace}.{topic_name}"

headers = {
    "Content-Type": "application/vnd.kafka.avro.v2+json",
    "Accept": "application/vnd.kafka.v2+json",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)